# How to use LiteLLM and Smolagents with Amazon Nova models

In this tutorial we are going to:

1. Configure a LiteLLM load-balancer with Amazon Nova models. 
2. Configure LiteLLM with the Hugging Face smolagents agentic framework with Cross-Region Inference with Amazon Nova.

Before running this notebook:
- Make sure you have enabled the models Amazon Micro and Pro in your AWS Console in us-east-1 and us-west-2. For more informations about the process check the following link: 
https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html
- Configure your AWS credentials: https://docs.aws.amazon.com/sdk-for-java/v1/developer-guide/setup-credentials.html
- AWS IAM permissions required: https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles-prereq.html

## Install requirements

In [1]:
%pip install litellm==1.59.8 smolagents==1.6.0 boto3==1.36.8 --quiet


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. LiteLLM and Amazon Nova
LiteLLM is an open-source framework which handles loadbalancing, fallbacks and spend tracking across 100+ LLMs. We are going to use Amazon Nova Micro and Pro via the LiteLLM Router

In [2]:
import os
from litellm import Router

# aws region
os.environ["AWS_REGION_NAME"] = "us-east-1"

/efs/dev-noteb/litellm/smol-agent-nova/litellm/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


 Each of the models will have different Requests Per Minute (RPM) configurations, feel free to modify according to your usage. 

In [3]:
model_list = [
    {
        "model_name": "nova_models",
        "litellm_params": { 
            "model": "bedrock/amazon.nova-micro-v1:0",          #nova micro model id
            "aws_region_name": os.getenv("AWS_REGION_NAME"),    #aws region
            #TPM and RPM configuration - this is going to be used by the router
            "tpm": 400000,
            "rpm": 200,
        },
    }, 
    {
        "model_name": "nova_models", 
        "litellm_params": {
            "model": "bedrock/amazon.nova-pro-v1:0",
            "aws_region_name": os.getenv("AWS_REGION_NAME"),    #aws region
            #TPM and RPM configuration - this is going to be used by the router
            "tpm": 400000,
            "rpm": 100,
        }
    }
]

Here is our router:

In [4]:
router = Router(
    model_list=model_list,
    routing_strategy="usage-based-routing-v2",
    enable_pre_call_checks=True,
    num_retries=1,
    set_verbose=False,
    allowed_fails=1,
    cooldown_time=100,
    disable_cooldowns=False
)

## Testing router
It is time to test our router.

In [5]:
response = await router.acompletion(
    model="nova_models",
    messages=[{"role": "user", "content": "Hey, how's it going?"}],
    temperature=0.1,
)

print(response.choices[0].message.content)

Hey there! I'm doing great, thanks for asking. How can I assist you today? Whether you have questions, need some information, or just want to chat, I'm here for you. What's on your mind?


## 2. Hugging Face Smolagents
The Hugging Face smolagents is an open-source agentic framework. We are going to use the Smolagents LiteLLM integration to invoke Amazon Nova Pro model. Instead of using the LiteLLM load-balancing, this time we will be using the Amazon Bedrock Cross-Region Inference, which will automatically route our inference requests between *us-east-1 and us-west-2*.

In [6]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel

model = LiteLLMModel(model_id="bedrock/us.amazon.nova-pro-v1:0")

# The DuckDuckGoSearchTool enables our agent to search and retrieve results from the DuckDuckGo search engine. 
# The tools calls will be formulated by the LLM in code format, then parsed and executed.
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")

/efs/dev-noteb/litellm/smol-agent-nova/litellm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many seconds would it take for a leopard at full speed to run through Pont des Arts?                        │
│                                                                                                                 │
╰─ LiteLLMModel - bedrock/us.amazon.nova-pro-v1:0 ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  pont_des_arts_length = web_search(query="length of Pont des Arts")                                               
  print("Length of Pont des Arts:", pont_des_arts_length)                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Length of Pont des Arts: ## Search Results

[Pont des Arts - Wikipedia](https://en.wikipedia.org/wiki/Pont_des_Arts)
The Pont des Arts (French pronunciation: [pɔ̃ dez‿aʁ]) or Passerelle des Arts ([pasʁɛl -]) is a pedestrian bridge in
Paris which crosses the River Seine. It links the Institut de France and the central square (cour carrée) of the 
Palais du Louvre, (which had been termed the "Palais des Arts" under the First French Empire).

[Pont des Arts, Paris, France - LatLong](https://www.latlong.net/place/pont-des-arts-paris-france-32945.html)
Pont des Arts, Paris, France Latitude and longitude coordinates are: 48.858334, 2.337369. Pont des Arts is a scenic
pedestrian bridge over the Seine River in Paris, France. The bridge connects the Musée du Louvre and the Institut 
de France. It has a length of 155 m (509 ft) and a width of 11 m (36 ft). It was constructed between 1981 and 1984.
The bridge is part of the UNESCO World Heritage ...

[Exploring the History of the Pont des Arts in 
Paris](https://www.parisartstravel.com/exploring-the-history-of-the-pont-des-arts-in-paris/)
The original Pont des Arts was constructed entirely from wood and featured nine arches with a total length of 155 
meters. It quickly became a popular meeting point for artists, writers, and intellectuals due to its proximity to 
the vibrant Latin Quarter.

[The Unique Architecture of the Pont des Arts in 
Paris](https://www.parisartstravel.com/the-unique-architecture-of-the-pont-des-arts-in-paris/)
What sets the Pont des Arts apart from other bridges in Paris is its distinctive design. The bridge is composed of 
nine arches and spans approximately 155 meters in length.

[A Complete Guide to the Pont des Arts in Paris - 
TripSavvy](https://www.tripsavvy.com/a-complete-guide-to-the-pont-des-arts-in-paris-4584846)
One of the loveliest of Paris' many bridges, the Pont des Arts is a photogenic delight. It's appeared in countless 
films, including one that shares its name. Connecting the central courtyard of the Louvre Palace on one side of the
Seine River to the prestigious Institut de France on the other, the bridge seems to embody the city at its most 
elegant.

[10 things to know about the Pont des Arts in Paris. - 
ParisCadenas.com](https://www.pariscadenas.com/articles/read/art-bridge)
The Pont des Arts is a wooden pedestrian bridge that connects the two sides of the Seine in Paris, between the 
Louvre and the rue de Seine. It was built between 1802 and 1804 by architect Louis-Alexandre de Cessart and 
engineer Jean-Rodolphe Perronet. The Pont des Arts is the first wooden bridge to be built over the Seine in Paris.

[History of the Pont des Arts in Paris • Come to 
Paris](https://www.cometoparis.com/paris-guide/paris-monuments/pont-des-arts-s959)
Spanning the Seine in central Paris, the pedestrian bridge known as the Pont des Arts has an international 
reputation as the bridge of romance. It was the first metal bridge to be built in the city and connects the Quai de
Conti at the Institut de France, with the Quai des Tuileries, where the Louvre Museum is situated. The history of 
an icon The first metal bridge in Paris was built very ...

[Learn everything about the history of Pont des Arts in Paris](https://sainte-chapelle.co/pont-des-arts/)
Learn everything about the history of Pont des Arts in Paris While crossing the River Seine, you will spot the most
photogenic bridge in France, Pont des Arts. The Cultural Ministry, in 1975, designated this bridge as a historical 
monument. Due to its structural beauty and popularity, you can see it in many shows and films.

[Pont des Arts Map - Bridge - Paris, Île-de-France, France - Mapcarta](https://mapcarta.com/29198828)
Pont des Arts The Pont des Arts or Passerelle des Arts is a pedestrian bridge in Paris which crosses the River 
Seine. It links the Institut de France and the central square of the Palais du Louvre.

[10 Picturesque Facts About The Pont Des Arts](https://art-facts.com/pont-des-

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  import math                                                                                                      
                                                                                                                   
  # Convert top speed from km/h to m/s                                                                             
  leopard_top_speed_ms = 58 * 1000 / 3600                                                                          
                                                                                                                   
  # Calculate the time in seconds                                                                                  
  time_seconds = 155 / leopard_top_speed_ms                                                                        
                                                                                                                   
  # Round the result to 2 decimal places                                                                           
  time_seconds = round(time_seconds, 2)                                                                            
                                                                                                                   
  final_answer(time_seconds)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 9.62

[Step 2: Duration 4.30 seconds| Input tokens: 10,473 | Output tokens: 371]

9.62

## Adding custom tools
Lets add a second tool to our agent, a custom tool which return the weather details using a static response.

In [7]:
from smolagents import tool

@tool
def get_weather(location: str) -> str:
    """
    Get weather in the next days at given location.

    Args:
        location: the location
    """
    return "The weather is freezing cold with torrential rains and temperatures below -10°C"


agent = CodeAgent(tools=[DuckDuckGoSearchTool(), get_weather], model=model)

print("ToolCallingAgent:", agent.run("What's the weather like in Paris?"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the weather like in Paris?                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - bedrock/us.amazon.nova-pro-v1:0 ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  paris_weather = get_weather(location="Paris")                                                                    
  print(paris_weather)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The weather is freezing cold with torrential rains and temperatures below -10°C

Out: None

[Step 0: Duration 1.07 seconds| Input tokens: 2,205 | Output tokens: 57]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The weather in Paris is freezing cold with torrential rains and temperatures below -10°C.")        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The weather in Paris is freezing cold with torrential rains and temperatures below -10°C.

[Step 1: Duration 1.04 seconds| Input tokens: 4,568 | Output tokens: 121]

ToolCallingAgent: The weather in Paris is freezing cold with torrential rains and temperatures below -10°C.
